# Шаблон для проекта по A/B - тестам.

### `chat_id ` из Телеграма:

In [ ]:
511551989

------------------

##  <ins>Задание 1.</ins>

Контекст
Мы считаем, что наш продукт достаточно дорогой для клиента.
У нас есть гипотеза, что уменьшение цены продукта
позволит повысить частоту продаж
и суммарно увеличит доходность продукта.
        
Тест
Мы проводим тест,
в рамках которого контролем является
продажа продукта со старой ценой,
а тестом - продажа продукта с новой ценой.
Целью этого теста
является проверка гипотезы роста доходности
нашего продукта из-за уменьшения тарифа.

Статистические вводные
Уровень значимости 5%.
Мы хотели бы различать 5%-ное
изменение целевой метрики
с вероятностью 80%.

### 0. Запосить у бота в Телеграме условие командой `/get_project1_desc`.

### 1. Необходимо выбрать метрику и привести аргументацию.


Выбранная метрика средний NPV по всем клиентам.
Выбрав эту метрику можно определить, что снижение цены на продукт даст статзначимый результат, так как этот показатель учитывает, как доход от клиента, так и расход от его привелечения

### 2. Альтернатива в критерии.

$H_0$- средний NPV не поменялось

$H_1$ - средний NPV в тесте больше среднего NPV в контроле

### 3.1. Каков размер выборки? Привести аргументацию и написать как получилось то или иное число.

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab
import scipy
import scipy.stats as stats
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import ttest_ind, mannwhitneyu
#from tqdm.notebook import tqdm
data = pd.read_csv("hist_telesales.csv")
df_1


,ID,Флаг дозвона,Флаг продажи,Расходы,PV,NPV
0,0,1,0,90,0,-90
1,1,0,0,5,0,-5
2,2,0,0,68,0,-68
3,3,1,0,22,0,-22
4,4,1,0,22,0,-22
...,...,...,...,...,...,...
72156,72156,1,1,577,1346,769
72157,72157,0,0,8,0,-8
72158,72158,0,0,23,0,-23
72159,72159,0,0,4,0,-4


Выбираем правосторонюю альтернативу

A = 0.05

B = 0.2 (1 - 0.8)

MDE = 0.05 * data['PV'].mean()

Дисперсия NPV = np.var(data['NPV'])

In [23]:
n = (2 * np.var(data['NPV'])*(scipy.stats.norm.ppf(0.95) - scipy.stats.norm.ppf(0.2))**2) / (0.05 * data['NPV'].mean())**2
print(round(n))

28671


### 3.2. Необходимо запросить у бота в Телеграме выборку полученного размера командой `/get_project1_sample {размер выборки}` и вписать <ins>код выборки</ins>:

Код выборки: 37510


In [25]:
control_data = pd.read_csv('Контроль.csv')
test_data = pd.read_csv('Тест.csv')

In [27]:
scipy.stats.ttest_ind(control_data['NPV'], test_data['NPV'], alternative='less')

TtestResult(statistic=-1.5350078138361862, pvalue=0.0623937574216684, df=56432.0)

from scipy.stats import ttest_ind
scipy.stats.ttest_ind(control_data['NPV'], test_data['NPV'], test_data='less')


### 4. Принятие решения. Расписать подробно с аргументами.

**Вывод:**
Вычисленное  pvalue=0.0623937574216684 > alpha, следовательно - принимаем $H_0$.

Различие между выборочным средним NPV на тесте и на контроле статистически незначимое.



##  <ins>Задание 2.</ins>

**Контекст**

Мы провели конкурентный анализ
и поняли,
что тариф нашей услуги достаточно низкий.
Мы хотим попробовать повысить тариф,
и надеемся,
что снижение конверсии в подключение услуги
компенсируется ростом дохода с этой услуги.
        
**Тест**

Мы проводим тест,
в рамках которого мы разбиваем весь поток
на две части рановероятно,
где одной части (контролю) 
мы будем предлагать старый тариф услуги,
а второй части (тесту) - новый тариф услуги.
Целью этого теста
является увеличение доходности услуги.

**Статистические вводные**

Проверка однородности выбранного параметра
осуществяется с уровнем значимости 2%.
Уровень значимости 5%.
Мы хотели бы различать 8%-ное
изменение целевой метрики
с вероятностью 90%.

### 0. Запосить у бота в Телеграме условие командой `/get_project2_desc`.

### 1. Необходимо выбрать метрику и привести аргументацию.

In [38]:
data = pd.read_csv('hist_credit_card.csv')
data

,ID,Возраст,Доход клиента,Вероятность банкротства,Флаг утилизации счёта,Расходы,PV КК,PV услуги,NPV
0,0,19,21620.835463,0.138061,0,102,0,0,-102
1,1,27,24897.990863,0.035508,1,409,11686,1754,13031
2,2,50,23989.526947,0.098793,0,16,0,0,-16
3,3,18,38442.409756,0.365661,1,788,13738,1578,14528
4,4,24,21291.521612,0.036909,1,1048,6594,2213,7759
...,...,...,...,...,...,...,...,...,...
123250,123250,32,26099.633927,0.110756,0,47,0,0,-47
123251,123251,20,24579.749275,0.113920,1,594,14268,1672,15346
123252,123252,36,34062.902531,0.247122,1,77,5950,2017,7890
123253,123253,67,24609.838522,0.020752,1,279,6278,1847,7846


### 2. Альтернатива в критерии.

### 3. Нужно выбрать параметр(-ы), влияющий(-ие) на целевую метрику. Привести аргументацию.

### 4.1. Каков размер выборки? Привести аргументацию и написать как получилось то или иное число.

### 4.2. Необходимо запросить у бота в Телеграме выборку полученного размера командой `/get_project2_sample {размер выборки}` и вписать <ins>код выборки</ins>:

### 5. Проверка на однородность, применение критерия. Принятие решения. Расписать подробно с аргументами.